__Title:__ Mini-Lab: Logistic Regression and SVMs  
__Authors:__ Butler, Derner, Holmes, Traxler  
__Date:__ 1/22/23 

## Rubric
You are to perform predictive analysis (classification) upon a data set: model the dataset using 
methods we have discussed in class: logistic regression & support vector machines and making 
conclusions from the analysis. Follow the CRISP-DM framework in your analysis (you are not 
performing all of the CRISP-DM outline, only the portions relevant to the grading rubric outlined 
below). This report is worth 10% of the final grade. You may complete this assignment in teams 
of as many as three people. 
Write a report covering all the steps of the project. The format of the document can be PDF, 
*.ipynb, or HTML. You can write the report in whatever format you like, but it is easiest to turn in 
the rendered Jupyter notebook. The results should be reproducible using your report. Please 
carefully describe every assumption and every step in your report.
A note on grading: A common mistake I see in this lab is not investigating different input 
parameters for each model. Try a number of parameter combinations and discuss how the 
model changed. 
SVM and Logistic Regression Modeling 
 - [50 points] Create a logistic regression model and a support vector machine model for the 
classification task involved with your dataset. Assess how well each model performs (use 
80/20 training/testing split for your data). Adjust parameters of the models to make them 
more accurate. If your dataset size requires the use of stochastic gradient descent, then 
linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing 
logistic regression and linear support vector machines. For many problems, SGD will be 
required in order to train the SVM model in a reasonable timeframe. 
 - [10 points] Discuss the advantages of each model for each classification task. Does one 
type of model offer superior performance over another in terms of prediction accuracy? In 
terms of training time or efficiency? Explain in detail. 
 - [30 points] Use the weights from logistic regression to interpret the importance of different 
features for the classification task. Explain your interpretation in detail. Why do you think 
some variables are more important? 
 - [10 points] Look at the chosen support vectors for the classification task. Do these provide 
any insight into the data? Explain. If you used stochastic gradient descent (and therefore did 
not explicitly solve for support vectors), try subsampling your data to train the SVC model—
then analyze the support vectors from the subsampled dataset.

__CRISP-DM__
 - Business understanding – What does the business need?
 - Data understanding – What data do we have / need? Is it clean?
 - Data preparation – How do we organize the data for modeling?
 - Modeling – What modeling techniques should we apply?
 - Evaluation – Which model best meets the business objectives?
 - Deployment – How do stakeholders access the results?

 Source: [Hotz, 2023](https://www.datascience-pm.com/crisp-dm-2/)



__Buisness Understanding__  
What features are most important in predicting which flights will be delayed?

__Data Understanding__  
We have a dataset of over 200,000 flights and 60 features from US Carriers in 2021. Approximately 33% of flights are delayed. There are some features with null values. These features are largely associated with columns that will be removed for the simple reason that they would not be knowable prior to the flight. We will have to prune the dataset to only include knowable features. Some of the features are highly correlated because they represent very similar things (e.g. scheduled departure time(CRSDepTime) vs. actual departure time(DepTime)). While most of these will be removed because they are not knowable, the remaining will be chosen based on the completeness of the data. The remaining issue to address is multi-colinearity. The majority of correlated features will be removed in the above steps. The remaining will again be chosen by the completeness of their data. Any remaining observations with incomplete data will be removed.

__Data Preparation__  
All categorical features will require one hot encoding. Some will not be usable due to the number of levels compared to the number of observations available.

The continous features will be normalized to reduce the influence of features with large values.

__Modeling__  
We will be comparing Logistic Regression and Support Vector Machines in this notebook. Each model will use the same training and testing datasets.

__Evaluation__  
The overall performance of each model will be evaluated by their respective accuracy, sensitivity, and specificity. We have concluded that it is more important to accurately predict the true occurance of delayed flights. For this reason, we will use sensitivity as the primary metric to compare and evaluate each model.

__Deployment__  
The findings of our study, including important features and their weights, can be found in the conclusion section of this rendered Jupyter notebook.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn import metrics

pd.set_option('display.max_columns', None)

In [ ]:
# Dataset
url = 'https://github.com/cdholmes11/MSDS-7331-ML1-Labs/blob/main/Lab-1_Visualization_DataPreprocessing/data/Combined_Flights_2021_sample.csv?raw=true'
flight_data_df = pd.read_csv(url, encoding = "utf-8")

In [ ]:
# New Features
flight_data_df['Delayed'] = np.where(flight_data_df['DepDelayMinutes'] > 0, 1, 0)
flight_data_df['Dif_Oper'] = np.where(flight_data_df['DOT_ID_Marketing_Airline'] == flight_data_df['DOT_ID_Operating_Airline'], 0, 1)

The original dataset doesn't have a feature for binary classification of delayed status. We've added one based on the DepDelayMinutes field. Delayed flights are coded as 1 and not delayed as 0.

Marketing Airline and Operatin Airline are often the same. One will likely be dropped from the model due to multi-colinearity. To maintain the important information from both columns, we've added a classification feature for when the flight is marketed by one airline and operated by another. Flights with different operators are coded as 1 and 0 for flights with the same marketing airline.

In [ ]:
# Dataset Shape
flight_data_df.shape

In [ ]:
# Delayed Frequency
delayed_df = pd.DataFrame(flight_data_df['Delayed'].value_counts()).reset_index()
delayed_df.columns = ['Delayed', 'Count']
delayed_df['Frequency'] = round(delayed_df['Count'] / sum(delayed_df['Count']) * 100, 2)
delayed_df

In [ ]:
# Knowable features grouped by data type
cat_features = ['Airline', 'Dest', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline', 'Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline', 'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'Operating_Airline', 'Origin', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'Tail_Number']
cont_features = ['CRSArrTime', 'CRSDepTime', 'CRSElapsedTime', 'Distance']
ord_features = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Delayed', 'Dif_Oper']
date_feature = ['FlightDate']

In [ ]:
# Trimmed Dataset to knowable features
flight_delay_df = flight_data_df[cat_features + cont_features + ord_features + date_feature]
flight_delay_df.shape

In [ ]:
# Correlation Plot using Plotly
flight_corr = flight_delay_df.corr()

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x = flight_corr.columns,
        y = flight_corr.index,
        z = np.array(flight_corr),
        text=flight_corr.values,
        texttemplate='%{text:.2f}' #set the size of the text inside the graphs
    )
)

fig.update_layout(
    title='Airline Feature Correlation',
    autosize=False,
    width=1000,
    height=600
)

fig.show()

In [ ]:
# Features converted to corresponding group type
flight_data_df[cat_features] = flight_data_df[cat_features].astype('category')
flight_data_df[ord_features] = flight_data_df[ord_features].astype(np.int64)
flight_data_df[cont_features] = flight_data_df[cont_features].astype(np.int64)
flight_data_df['FlightDate'] = pd.to_datetime(flight_data_df['FlightDate']).dt.date

In [ ]:
# Tail Number Evaluation
flight_delay_df['Tail_Number'].describe()

In [ ]:
# Tail Number Delay Frequency
tail_df = pd.pivot_table(flight_delay_df, values='Distance', index =['Tail_Number'], aggfunc = 'count', columns='Delayed').reset_index()
tail_df.columns = ['Tail_Number', 'On-Time', 'Delayed']
tail_df['Total'] = tail_df['Delayed'] + tail_df['On-Time']
tail_df['Frequency'] = round((tail_df['Delayed'] / tail_df['Total'] * 100),2)

tail_df[tail_df['Total'].notnull()].sort_values(by=['Delayed'], ascending=False)

Many categorical features have a corresponding numeric feature. Wherever possible, we've decided to utilize the numeric representation of these features.

cat_remove = ['Airline', 'Dest', 'DestCityName', 'DestState', 'DestStateName', 
    'DOT_ID_Operating_Airline', 'Flight_Number_Operating_Airline', 'IATA_Code_Marketing_Airline',
    'IATA_Code_Operating_Airline', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
    'Operating_Airline', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateName', 'DestAirportSeqID',
    'OriginAirportSeqID']


In [ ]:
# Features to remove
cat_remove = ['Airline', 'Dest', 'DestCityName', 'DestState', 'DestStateName', 
    'DOT_ID_Operating_Airline', 'Flight_Number_Operating_Airline', 'IATA_Code_Marketing_Airline',
    'IATA_Code_Operating_Airline', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
    'Operating_Airline', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateName', 'DestAirportSeqID',
    'OriginAirportSeqID', 'Tail_Number', 'DestCityMarketID']
cont_remove = ['CRSArrTime', 'CRSElapsedTime']
ord_remove = ['Quarter']

In [ ]:
# Removing features from category groups
for n in cat_remove:
    if n in cat_features:
        cat_features.remove(n)

for n in cont_remove:
    if n in cont_features:
        cont_features.remove(n)

for n in ord_remove:
    if n in ord_features:
        ord_features.remove(n)


In [ ]:
# Creating new dataset with updated feature lists
flight_delay_df = flight_data_df[cat_features + cont_features + ord_features]
flight_delay_df.shape

Tail Number has over 5700 unique values. One Hot Encoding Tail Number will add too many features to the dataset, for the given observation count. Additionaly, future datasets will likely contain different tail numbers. Practically, this makes tail number an unrealistic feature to include in the model. For the curious, we've provided a table of tail numbers sourted by the number of times delayed and provided their relative freqency of being delayed. There are clearly some aircraft that are much more likely to be delayed than others.

In [ ]:
flight_delay_df[cat_features].describe()

In [ ]:
flight_delay_df.info()

In [ ]:
# Train/Test Split
X = flight_delay_df.drop(['Delayed'], axis=1)
Y = flight_delay_df['Delayed']

In [ ]:
# One Hot Encoding
onehot_encoder = OneHotEncoder(drop='first', sparse_output=True)
label_encoder = LabelEncoder()
scaler = StandardScaler(with_mean=False)

X = scaler.fit_transform(X)
X = onehot_encoder.fit_transform(X)
Y = label_encoder.fit_transform(Y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=110)

## Logistic Regression

In [ ]:
# Basic Logistic Regression
logmod = LogisticRegression(random_state=10, max_iter=5000)
log_fit = logmod.fit(X_train, Y_train)
y_pred = logmod.predict(X_test)

# Model Performance
print('Basic Score: ', logmod.score(X_train, Y_train))
print(classification_report(Y_test, y_pred))

In [ ]:
# Source: https://www.w3schools.com/python/python_ml_confusion_matrix.asp
confusion_matrix = confusion_matrix(Y_test, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

With a precision of 56%, the untuned logistic regression model is an improvement over the delayed frequency of the sample. That is to say, it's true positive rate is greater than 33%. While the overall model accuracy of 70% is barely an improvement from the baseline delayed sample frequency, precision is far more important than accuracy for this model.

In our reasearch, we have found the initial feature size, after One Hot Encoding, to be too resource entensive to be practical. Moving forward, we will use the basic logistic model to find the minimum features resulting in similar results. This will greatly reduce the processing requirements for future models. 

In [ ]:
# Feature selection
prec_list = []
i = 50
while i < 5000:
    logmod_feture = LogisticRegression(random_state=10, max_iter=5000)
    selector = SelectFromModel(logmod_feture, max_features=i).fit(X_train, Y_train)
    
    X_train_new = selector.transform(X_train)
    X_test_new = selector.transform(X_test)

    logmod_feture.fit(X_train_new, Y_train)
    y_pred_feature = logmod_feture.predict(X_test_new)

    prec_score = precision_score(Y_test, y_pred_feature, average='macro')
    prec_list.append([prec_score, i])

    i+=50

In [ ]:
# Plot Max Features vs. Precision
prec = pd.DataFrame(prec_list)
prec.columns = ['Precision', 'Max_Features']

max_prec = max(prec['Precision'])
best_max_feature = prec['Max_Features'][prec['Precision'] == max_prec]
best_max_feature = round(best_max_feature.item(), 2)
print(f'The Max Features that produces the best precision is {best_max_feature}')

fig = px.line(prec, x="Max_Features", y="Precision", title='Precision by Max_Features' , width=1000)
fig.add_vline(x=best_max_feature, line_color="red")
fig.show()

In [ ]:
logmod_trim = LogisticRegression(random_state=10, max_iter=5000)
selector = SelectFromModel(logmod_trim, max_features=1000).fit(X_train, Y_train)

X_train_new = selector.transform(X_train)
X_test_new = selector.transform(X_test)

In [ ]:
# Grid Search Param
param_grid = [    
    {'penalty' : ['l1', 'l2', 'none'],
    'C' : [0.01, 0.1, 1, 10, 100, 1000],
    'solver' : ['newton-cholesky','sag','saga'],
    'max_iter' : [2000, 5000]
    }
]

In [ ]:
# Optimized Logistic Regression
logmod_cv = LogisticRegression(random_state=10)
clf = GridSearchCV(
    logmod_cv,
    param_grid = param_grid, 
    cv=None ,
    verbose=False,
    n_jobs=-1, 
    scoring='precision',
    refit=True
    )
best_clf = clf.fit(X_train_new, Y_train)

In [ ]:
best_clf.best_params_

In [ ]:
best_clf.best_score_

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

log_final = LogisticRegression(
    C=0.1,
    max_iter=2000,
    penalty= 'l2',
    solver='newton-cholesky'
)
log_final.fit(X_train_new, Y_train)

scores = cross_validate(log_final, X_train_new, Y_train, scoring='precision', cv=10, return_train_score=True)
y_pred_final = cross_val_predict(log_final, X_test_new, Y_test, cv=10)

print(cross_val_score(log_final, X_test_new, Y_test, cv=10))
print(classification_report(Y_test, y_pred_final))

After tuning the hyperparameters, we were able to improve our precision by 6%. Since l2 is the default penalty, the two primary changes were to C and the solver. A reduced C from 1 to 0.1 and changing the solver from lbfgs to newton-cholesky provided the above results. This is in alignment with the scikit-learn documentation that states 'newton-cholesky' is a "good choice for one-hot encoded categoircal features with rare categories" (Sklearn.Linear_Model.LogisticRegression, n.d.).

## Support Vector Machines

In [ ]:
svc_model = svm.SVC()

svc_model.fit(X_train, Y_train)

y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:", metrics.accuracy_score(Y_test, y_pred))

## Model Advantages

## Feature Importance

In [ ]:
importance = log_final.coef_[0]
data = {'Coef': importance,'Feature': X_train_new[0]}
df2 = pd.DataFrame(data)
df2

In [ ]:
importance

## SVC Insights

## Sources

1. Hotz, N. (2023, January 19). What is CRISP DM? Data Science Process Alliance. https://www.datascience-pm.com/crisp-dm-2/
2. sklearn.linear_model.LogisticRegression. (n.d.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html